# Data driven approach


## monetray model +  yield curve

multistep: multistep forecast  
m: monthly

### dependent variable

- "DEXJPUS","fred"

### explanatory variables
- "MANMM101USM189S","fred"
- "MANMM101JPM189S","fred"
- "T1YFF","fred"
- "T10YFF","fred"
- "T10Y2Y","fred"
- "T10Y3M","fred"

In [1]:
%matplotlib inline
import datetime
import pandas_datareader.data as web
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.stats import norm
from statsmodels.tsa.api import VECM
start="1971/1/1";end='2019/12/31'
from dateutil.relativedelta import relativedelta
def ncdf(x): #t-distribution
    if x>=0:
        y=(1-norm.cdf(x))
    else:
        y=norm.cdf(x)
    return y

In [2]:
e =web.DataReader("DEXJPUS","fred",start,end)
m1 =web.DataReader("MANMM101USM189S","fred",start,end)/1000000000#w
m2 =web.DataReader("MANMM101JPM189S","fred",start,end)/1000000000#w
s1 =web.DataReader("T1YFF","fred",start,end)
s2 =web.DataReader("T10YFF","fred",start,end)
s3 =web.DataReader("T10Y2Y","fred",start,end)
s4 =web.DataReader("T10Y3M","fred",start,end)


In [3]:
FRQ=['MS']
interval=1
for i in range(len(FRQ)):
    ee=e.copy().resample(FRQ[i]).last()
    ee.columns=['x']
    mm1=m1.copy().resample(FRQ[i]).last()
    mm1.columns=['x']
    mm2=m2.copy().resample(FRQ[i]).last()
    mm2.columns=['x']
    ss1=s1.copy().resample(FRQ[i]).last()
    ss1.columns=['x']
    ss2=s2.copy().resample(FRQ[i]).last()
    ss2.columns=['x']
    ss3=s3.copy().resample(FRQ[i]).last()
    ss3.columns=['x']
    ss4=s4.copy().resample(FRQ[i]).last()
    ss4.columns=['x']
    tsd=pd.concat([ee,mm1,mm2,ss2,ss2],axis=1)
    tsd.columns=['ee','m1','m2','s1','s2']
    #tsd=tsd.fillna(0)
    idx=e.index[-1]-relativedelta(months=2)
    tsd=tsd.copy().loc[:idx]
    print(i,FRQ[i],tsd.tail(5))
    if i==0:
        tsd1=tsd
    elif i==1:
        tsd2=tsd
    else:
        tsd3=tsd

pstart=datetime.date(1983,1,1)
#pend=datetime.date(1995,12,31)        
pend=datetime.date(2020,12,31)        

0 MS                 ee      m1        m2    s1    s2
DATE                                            
2019-06-01  107.84  3832.9  793923.2 -0.40 -0.40
2019-07-01  108.58  3858.3  795860.8 -0.38 -0.38
2019-08-01  106.30  3853.4  800558.2 -0.63 -0.63
2019-09-01  108.11  3903.3  804952.1 -0.22 -0.22
2019-10-01  108.09  3923.4  807891.6  0.11  0.11


In [5]:
def backtest_ols_dp(tsd,tw,pt):#tw time window pt:predicting time
    pred=[]
    real0=[]
    real9=[]
    tsd0=np.log(tsd.iloc[:,:3]).copy()
    for i in range(tw+1,len(tsd)+1-1):
        yy=tsd0.iloc[i-tw:i:1,0]
        y0=tsd0.iloc[i-tw-1:i:1,0].diff().dropna()
        x0=(tsd0.iloc[i-tw-1:i:1,1]-tsd0.iloc[i-tw-1:i:1,2]).diff().dropna()
        x00=tsd.iloc[i-tw-1:i:1,3].diff().dropna()
        x0=pd.concat([x0,x00],axis=1)
        x0=sm.add_constant(x0.values)
        res= sm.OLS(y0, x0).fit()
        real0.append(np.exp(tsd0.iloc[i-1:i,0][-1]))
        if i<len(tsd)+1-pt:
            x9=(tsd0.iloc[i-1:i+pt:1,1]-tsd0.iloc[i-1:i+pt:1,2]).diff().dropna()
            x99=(tsd.iloc[i-1:i+pt:1,3]).diff().dropna()
            x9=pd.concat([x9,x99],axis=1)
            if pt==1:
                x9=np.insert(x9.values,0,1)
            else:
                x9=sm.add_constant(x9.values)
            pred0=res.predict(x9)[-1:]
            pred.append(np.exp(pred0+yy.iloc[-1]))
            real9.append(np.exp(tsd0.iloc[i+pt-1:i+pt:1,0].values))
    return real0,real9,pred

def performance_eval(real0,real9,pred):
    p0=pd.DataFrame(real0)
    p9=pd.DataFrame(real9)
    pp=pd.DataFrame(pred)
    p0_=p0[:len(p9)]
    pp_=pp[:len(p9)]
    wrs=p0_[(p9<p0_)&(pp_<p0_)].count()
    wrl=p0_[(p9>p0_)&(pp_>p0_)].count()
    wr=float((wrs+wrl)/p0_.count())
    #wrp=tcdf(float((wr-0.5)/np.sqrt(0.25/p0_.count())),p0_.count())[0]
    wrp=ncdf(float((wr-0.5)/np.sqrt(0.25/p0_.count())))
    d_x=(pp_-p9)**2
    d_rw=(p9-p0_)**2
    d=d_x-d_rw
    d_=float(d.mean())
    #dm=tcdf(float(d_/np.sqrt(((d-d_)**2).sum()/len(d)**2)),p0_.count())[0]
    dm=ncdf(float(d_/np.sqrt(((d-d_)**2).sum()/len(d)**2)))
    rmse=float(d_x.mean()/d_rw.mean())
    return rmse,dm,wr,wrp

def performance_report(real0,real9,pred):
    ab,dm,wr,wrp=performance_eval(real0,real9,pred)
    print('{0:20s}{1:5d} mse_r {2:7.4f} mse_p {3:7.2f} wr {4:7.4f} wrp {5:7.2f} len {6:4d}'
          .format(model,isp,ab,dm,wr,wrp,len(real9)))    

In [6]:
model='first difference OLS'
pstart=datetime.date(1983,1,1)
pend=datetime.date(1994,12,31)        
oosp=3
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*4,oosp*50,12):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

first difference OLS   12 mse_r  1.3365 mse_p    0.14 wr  0.5286 wrp    0.25 len  140
first difference OLS   24 mse_r  1.0012 mse_p    0.49 wr  0.5000 wrp    0.50 len  140
first difference OLS   36 mse_r  0.9776 mse_p    0.31 wr  0.5643 wrp    0.06 len  140
first difference OLS   48 mse_r  1.0016 mse_p    0.47 wr  0.5143 wrp    0.37 len  140
first difference OLS   60 mse_r  0.9951 mse_p    0.39 wr  0.5429 wrp    0.16 len  140
first difference OLS   72 mse_r  0.9947 mse_p    0.34 wr  0.5357 wrp    0.20 len  140
first difference OLS   84 mse_r  1.0038 mse_p    0.37 wr  0.4786 wrp    0.31 len  140
first difference OLS   96 mse_r  0.9824 mse_p    0.03 wr  0.5714 wrp    0.05 len  140
first difference OLS  108 mse_r  0.9736 mse_p    0.01 wr  0.5714 wrp    0.05 len  140
first difference OLS  120 mse_r  0.9718 mse_p    0.00 wr  0.5643 wrp    0.06 len  140
first difference OLS  132 mse_r  0.9708 mse_p    0.00 wr  0.5929 wrp    0.01 len  140
first difference OLS  144 mse_r  0.9828 mse_p    0.03 

In [7]:
model='first difference OLS'
pstart=datetime.date(1994,1,1)
pend=datetime.date(2007,12,31)        
oosp=3
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*4,oosp*50,12):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

first difference OLS   12 mse_r  1.2083 mse_p    0.01 wr  0.5427 wrp    0.14 len  164
first difference OLS   24 mse_r  1.1064 mse_p    0.02 wr  0.4634 wrp    0.17 len  164
first difference OLS   36 mse_r  1.0888 mse_p    0.02 wr  0.4451 wrp    0.08 len  164
first difference OLS   48 mse_r  1.0715 mse_p    0.01 wr  0.4024 wrp    0.01 len  164
first difference OLS   60 mse_r  1.0451 mse_p    0.05 wr  0.4756 wrp    0.27 len  164
first difference OLS   72 mse_r  1.0356 mse_p    0.05 wr  0.4268 wrp    0.03 len  164
first difference OLS   84 mse_r  1.0468 mse_p    0.04 wr  0.4268 wrp    0.03 len  164
first difference OLS   96 mse_r  1.0463 mse_p    0.06 wr  0.4390 wrp    0.06 len  164
first difference OLS  108 mse_r  1.0384 mse_p    0.10 wr  0.4756 wrp    0.27 len  164
first difference OLS  120 mse_r  1.0294 mse_p    0.12 wr  0.4695 wrp    0.22 len  164
first difference OLS  132 mse_r  1.0244 mse_p    0.11 wr  0.5122 wrp    0.38 len  164
first difference OLS  144 mse_r  1.0092 mse_p    0.28 

In [8]:
model='first difference OLS'
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)        
oosp=3
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*4,oosp*50,12):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

first difference OLS   12 mse_r  1.0354 mse_p    0.30 wr  0.5435 wrp    0.15 len  138
first difference OLS   24 mse_r  0.9707 mse_p    0.30 wr  0.5725 wrp    0.04 len  138
first difference OLS   36 mse_r  0.9595 mse_p    0.22 wr  0.5507 wrp    0.12 len  138
first difference OLS   48 mse_r  0.9917 mse_p    0.43 wr  0.5507 wrp    0.12 len  138
first difference OLS   60 mse_r  1.0140 mse_p    0.39 wr  0.5290 wrp    0.25 len  138
first difference OLS   72 mse_r  0.9969 mse_p    0.47 wr  0.5290 wrp    0.25 len  138
first difference OLS   84 mse_r  0.9939 mse_p    0.43 wr  0.5217 wrp    0.30 len  138
first difference OLS   96 mse_r  1.0017 mse_p    0.48 wr  0.5725 wrp    0.04 len  138
first difference OLS  108 mse_r  0.9942 mse_p    0.40 wr  0.5580 wrp    0.09 len  138
first difference OLS  120 mse_r  0.9913 mse_p    0.25 wr  0.5870 wrp    0.02 len  138
first difference OLS  132 mse_r  0.9983 mse_p    0.45 wr  0.5290 wrp    0.25 len  138
first difference OLS  144 mse_r  0.9932 mse_p    0.29 

In [9]:
model='first difference OLS'
pstart=datetime.date(1983,1,1)
pend=datetime.date(1994,12,31)        
oosp=12
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*1,oosp*50,12):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

first difference OLS   12 mse_r  0.9997 mse_p    0.50 wr  0.5802 wrp    0.03 len  131
first difference OLS   24 mse_r  0.9768 mse_p    0.05 wr  0.5954 wrp    0.01 len  131
first difference OLS   36 mse_r  0.9802 mse_p    0.04 wr  0.5725 wrp    0.05 len  131
first difference OLS   48 mse_r  1.0001 mse_p    0.49 wr  0.5344 wrp    0.22 len  131
first difference OLS   60 mse_r  0.9913 mse_p    0.05 wr  0.6031 wrp    0.01 len  131
first difference OLS   72 mse_r  0.9988 mse_p    0.41 wr  0.6412 wrp    0.00 len  131
first difference OLS   84 mse_r  0.9953 mse_p    0.13 wr  0.6718 wrp    0.00 len  131
first difference OLS   96 mse_r  0.9859 mse_p    0.00 wr  0.7176 wrp    0.00 len  131
first difference OLS  108 mse_r  0.9811 mse_p    0.00 wr  0.7328 wrp    0.00 len  131
first difference OLS  120 mse_r  0.9795 mse_p    0.00 wr  0.7634 wrp    0.00 len  131
first difference OLS  132 mse_r  0.9838 mse_p    0.00 wr  0.7405 wrp    0.00 len  131
first difference OLS  144 mse_r  0.9882 mse_p    0.00 

In [10]:
model='first difference OLS'
pstart=datetime.date(1994,1,1)
pend=datetime.date(2007,12,31)        
oosp=12
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*1,oosp*50,18):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

first difference OLS   12 mse_r  1.0420 mse_p    0.08 wr  0.5161 wrp    0.34 len  155
first difference OLS   30 mse_r  1.0755 mse_p    0.00 wr  0.3355 wrp    0.00 len  155
first difference OLS   48 mse_r  1.0467 mse_p    0.00 wr  0.3935 wrp    0.00 len  155
first difference OLS   66 mse_r  1.0264 mse_p    0.02 wr  0.4839 wrp    0.34 len  155
first difference OLS   84 mse_r  1.0216 mse_p    0.05 wr  0.4387 wrp    0.06 len  155
first difference OLS  102 mse_r  1.0099 mse_p    0.20 wr  0.4387 wrp    0.06 len  155
first difference OLS  120 mse_r  1.0084 mse_p    0.17 wr  0.5226 wrp    0.29 len  155
first difference OLS  138 mse_r  1.0029 mse_p    0.34 wr  0.4710 wrp    0.23 len  155
first difference OLS  156 mse_r  1.0045 mse_p    0.22 wr  0.4710 wrp    0.23 len  155
first difference OLS  174 mse_r  1.0075 mse_p    0.08 wr  0.4581 wrp    0.15 len  155
first difference OLS  192 mse_r  1.0052 mse_p    0.16 wr  0.4581 wrp    0.15 len  155
first difference OLS  210 mse_r  1.0010 mse_p    0.43 

In [11]:
model='first difference OLS'
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)        
oosp=12
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*1,oosp*50,24):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

first difference OLS   12 mse_r  0.9693 mse_p    0.16 wr  0.5969 wrp    0.01 len  129
first difference OLS   36 mse_r  0.9876 mse_p    0.29 wr  0.5891 wrp    0.02 len  129
first difference OLS   60 mse_r  1.0229 mse_p    0.08 wr  0.5039 wrp    0.46 len  129
first difference OLS   84 mse_r  1.0175 mse_p    0.06 wr  0.4651 wrp    0.21 len  129
first difference OLS  108 mse_r  1.0055 mse_p    0.23 wr  0.5426 wrp    0.17 len  129
first difference OLS  132 mse_r  1.0052 mse_p    0.19 wr  0.4806 wrp    0.33 len  129
first difference OLS  156 mse_r  1.0055 mse_p    0.13 wr  0.5116 wrp    0.40 len  129
first difference OLS  180 mse_r  1.0043 mse_p    0.17 wr  0.4729 wrp    0.27 len  129
first difference OLS  204 mse_r  0.9945 mse_p    0.10 wr  0.5426 wrp    0.17 len  129
first difference OLS  228 mse_r  0.9947 mse_p    0.12 wr  0.5194 wrp    0.33 len  129
first difference OLS  252 mse_r  0.9995 mse_p    0.46 wr  0.5349 wrp    0.21 len  129
first difference OLS  276 mse_r  0.9962 mse_p    0.22 

In [12]:
model='first difference OLS'
pstart=datetime.date(1983,1,1)
pend=datetime.date(1994,12,31)        
oosp=60
nn=len(tsd.loc[pstart:pend])
for isp in range(12,oosp*50,12):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

first difference OLS   12 mse_r  0.9797 mse_p    0.09 wr  0.6627 wrp    0.00 len   83
first difference OLS   24 mse_r  1.0010 mse_p    0.44 wr  0.6386 wrp    0.01 len   83
first difference OLS   36 mse_r  1.0049 mse_p    0.23 wr  0.6988 wrp    0.00 len   83
first difference OLS   48 mse_r  1.0013 mse_p    0.37 wr  0.6386 wrp    0.01 len   83
first difference OLS   60 mse_r  1.0040 mse_p    0.13 wr  0.6506 wrp    0.00 len   83
first difference OLS   72 mse_r  0.9970 mse_p    0.15 wr  0.8193 wrp    0.00 len   83
first difference OLS   84 mse_r  0.9915 mse_p    0.00 wr  0.8554 wrp    0.00 len   83
first difference OLS   96 mse_r  0.9879 mse_p    0.00 wr  0.9639 wrp    0.00 len   83
first difference OLS  108 mse_r  0.9873 mse_p    0.00 wr  0.9759 wrp    0.00 len   83
first difference OLS  120 mse_r  0.9898 mse_p    0.00 wr  0.9639 wrp    0.00 len   83
first difference OLS  132 mse_r  0.9900 mse_p    0.00 wr  0.9277 wrp    0.00 len   83
first difference OLS  144 mse_r  0.9890 mse_p    0.00 

In [13]:
model='first difference OLS'
pstart=datetime.date(1994,1,1)
pend=datetime.date(2007,12,31)        
oosp=60
nn=len(tsd.loc[pstart:pend])
for isp in range(12,oosp*50,12):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

first difference OLS   12 mse_r  1.2096 mse_p    0.00 wr  0.3551 wrp    0.00 len  107
first difference OLS   24 mse_r  1.1825 mse_p    0.00 wr  0.2804 wrp    0.00 len  107
first difference OLS   36 mse_r  1.1236 mse_p    0.00 wr  0.3084 wrp    0.00 len  107
first difference OLS   48 mse_r  1.0440 mse_p    0.04 wr  0.3832 wrp    0.01 len  107
first difference OLS   60 mse_r  1.0193 mse_p    0.17 wr  0.3925 wrp    0.01 len  107
first difference OLS   72 mse_r  1.0111 mse_p    0.24 wr  0.3364 wrp    0.00 len  107
first difference OLS   84 mse_r  1.0029 mse_p    0.42 wr  0.3925 wrp    0.01 len  107
first difference OLS   96 mse_r  1.0010 mse_p    0.47 wr  0.4112 wrp    0.03 len  107
first difference OLS  108 mse_r  1.0108 mse_p    0.23 wr  0.4112 wrp    0.03 len  107
first difference OLS  120 mse_r  1.0274 mse_p    0.01 wr  0.4112 wrp    0.03 len  107
first difference OLS  132 mse_r  1.0265 mse_p    0.01 wr  0.4299 wrp    0.07 len  107
first difference OLS  144 mse_r  1.0182 mse_p    0.03 

In [14]:
model='first difference OLS'
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)        
oosp=60
nn=len(tsd.loc[pstart:pend])
for isp in range(12,oosp*50,18):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

first difference OLS   12 mse_r  1.0104 mse_p    0.22 wr  0.5185 wrp    0.37 len   81
first difference OLS   30 mse_r  1.0282 mse_p    0.00 wr  0.2963 wrp    0.00 len   81
first difference OLS   48 mse_r  1.0295 mse_p    0.00 wr  0.2346 wrp    0.00 len   81
first difference OLS   66 mse_r  1.0194 mse_p    0.00 wr  0.3580 wrp    0.01 len   81
first difference OLS   84 mse_r  1.0149 mse_p    0.00 wr  0.3086 wrp    0.00 len   81
first difference OLS  102 mse_r  1.0158 mse_p    0.00 wr  0.2716 wrp    0.00 len   81
first difference OLS  120 mse_r  1.0152 mse_p    0.00 wr  0.3086 wrp    0.00 len   81
first difference OLS  138 mse_r  1.0167 mse_p    0.00 wr  0.2099 wrp    0.00 len   81
first difference OLS  156 mse_r  1.0157 mse_p    0.00 wr  0.1605 wrp    0.00 len   81
first difference OLS  174 mse_r  1.0135 mse_p    0.00 wr  0.1481 wrp    0.00 len   81
first difference OLS  192 mse_r  1.0118 mse_p    0.00 wr  0.1975 wrp    0.00 len   81
first difference OLS  210 mse_r  1.0119 mse_p    0.00 

In [15]:
def backtest_VECM(tsd,tw,pt,det):#tw time window pt:predicting time
    pred=[]
    real0=[]
    real9=[]
    tsd0=np.log(tsd.iloc[:,:3].copy())
    for i in range(tw,len(tsd)+1):
        y0=(tsd0.iloc[i-tw:i:1,0])
        x=tsd0.iloc[i-tw:i:1,1]-tsd0.iloc[i-tw:i:1,2]
        x0=tsd.iloc[i-tw:i:1,3]
        tsd00=pd.concat([y0,x,x0],axis=1)
        model = VECM(tsd00,k_ar_diff=0,deterministic=det)
        results = model.fit() 
        real0.append(np.exp(y0.iloc[-1]))
        pp=np.exp(results.predict(pt)[-1][0])
        pred.append(pp)
        if i<len(tsd)+1-pt:
            real9.append(tsd.iloc[i+pt-1,0])
    return real0,real9,pred

In [16]:
pstart=datetime.date(1983,1,1)
pend=datetime.date(1994,12,31)        
nn=len(tsd.loc[pstart:pend])
interval=1
model='VECM'
oosp=3
for isp in range(oosp*20,oosp*200,oosp*5):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'li')
        performance_report(real0,real9,pred)

VECM                   60 mse_r  1.0595 mse_p    0.00 wr  0.4255 wrp    0.04 len  141
VECM                   75 mse_r  1.0611 mse_p    0.00 wr  0.3972 wrp    0.01 len  141
VECM                   90 mse_r  1.0350 mse_p    0.01 wr  0.3759 wrp    0.00 len  141
VECM                  105 mse_r  1.0154 mse_p    0.06 wr  0.4113 wrp    0.02 len  141
VECM                  120 mse_r  1.0286 mse_p    0.03 wr  0.4610 wrp    0.18 len  141
VECM                  135 mse_r  1.0077 mse_p    0.28 wr  0.5532 wrp    0.10 len  141


In [17]:
pstart=datetime.date(1995,1,1)
pend=datetime.date(2007,12,31)        
nn=len(tsd.loc[pstart:pend])
interval=1
model='VECM'
oosp=3
for isp in range(oosp*20,oosp*200,oosp*10):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
        performance_report(real0,real9,pred)

VECM                   60 mse_r  1.0597 mse_p    0.17 wr  0.6078 wrp    0.00 len  153
VECM                   90 mse_r  1.1449 mse_p    0.05 wr  0.6405 wrp    0.00 len  153
VECM                  120 mse_r  1.1501 mse_p    0.04 wr  0.5752 wrp    0.03 len  153
VECM                  150 mse_r  1.1352 mse_p    0.01 wr  0.4837 wrp    0.34 len  153
VECM                  180 mse_r  1.0578 mse_p    0.01 wr  0.5033 wrp    0.47 len  153
VECM                  210 mse_r  1.1001 mse_p    0.01 wr  0.5490 wrp    0.11 len  153
VECM                  240 mse_r  1.0451 mse_p    0.04 wr  0.5556 wrp    0.08 len  153
VECM                  270 mse_r  1.0658 mse_p    0.01 wr  0.5425 wrp    0.15 len  153


In [18]:
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)        
nn=len(tsd.loc[pstart:pend])
interval=1
model='VECM'
oosp=3
for isp in range(oosp*20,oosp*200,oosp*10):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
        performance_report(real0,real9,pred)

VECM                   60 mse_r  1.0803 mse_p    0.05 wr  0.4460 wrp    0.10 len  139
VECM                   90 mse_r  1.0080 mse_p    0.31 wr  0.4964 wrp    0.47 len  139
VECM                  120 mse_r  1.0246 mse_p    0.05 wr  0.4892 wrp    0.40 len  139
VECM                  150 mse_r  1.0394 mse_p    0.01 wr  0.4676 wrp    0.22 len  139
VECM                  180 mse_r  0.9926 mse_p    0.34 wr  0.5396 wrp    0.18 len  139
VECM                  210 mse_r  1.0136 mse_p    0.27 wr  0.5180 wrp    0.34 len  139
VECM                  240 mse_r  0.9546 mse_p    0.01 wr  0.5899 wrp    0.02 len  139
VECM                  270 mse_r  0.9661 mse_p    0.01 wr  0.5036 wrp    0.47 len  139
VECM                  300 mse_r  0.9920 mse_p    0.26 wr  0.5468 wrp    0.14 len  139
VECM                  330 mse_r  0.9744 mse_p    0.03 wr  0.5540 wrp    0.10 len  139
VECM                  360 mse_r  0.9940 mse_p    0.33 wr  0.5683 wrp    0.05 len  139
VECM                  390 mse_r  0.9926 mse_p    0.27 

In [19]:
pstart=datetime.date(1983,1,1)
pend=datetime.date(1994,12,31)        
nn=len(tsd.loc[pstart:pend])
interval=1
model='VECM'
oosp=12
for isp in range(oosp*1,oosp*200,oosp*1):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'li')
        performance_report(real0,real9,pred)

VECM                   12 mse_r  1.3765 mse_p    0.01 wr  0.5379 wrp    0.19 len  132
VECM                   24 mse_r  1.1261 mse_p    0.02 wr  0.4015 wrp    0.01 len  132
VECM                   36 mse_r  1.0558 mse_p    0.00 wr  0.3636 wrp    0.00 len  132
VECM                   48 mse_r  1.0296 mse_p    0.00 wr  0.3409 wrp    0.00 len  132
VECM                   60 mse_r  1.0166 mse_p    0.16 wr  0.4773 wrp    0.30 len  132
VECM                   72 mse_r  1.0105 mse_p    0.17 wr  0.3864 wrp    0.00 len  132
VECM                   84 mse_r  1.0665 mse_p    0.03 wr  0.2424 wrp    0.00 len  132
VECM                   96 mse_r  1.0478 mse_p    0.00 wr  0.2955 wrp    0.00 len  132
VECM                  108 mse_r  1.0129 mse_p    0.03 wr  0.4470 wrp    0.11 len  132
VECM                  120 mse_r  1.0271 mse_p    0.00 wr  0.4697 wrp    0.24 len  132
VECM                  132 mse_r  1.0012 mse_p    0.46 wr  0.5758 wrp    0.04 len  132
VECM                  144 mse_r  0.9467 mse_p    0.00 

In [20]:
pstart=datetime.date(1995,1,1)
pend=datetime.date(2007,12,31)        
nn=len(tsd.loc[pstart:pend])
interval=1
model='VECM'
oosp=12
for isp in range(oosp*1,oosp*200,oosp*2):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
        performance_report(real0,real9,pred)

VECM                   12 mse_r  4.4195 mse_p    0.04 wr  0.5139 wrp    0.37 len  144
VECM                   36 mse_r  1.3544 mse_p    0.01 wr  0.5625 wrp    0.07 len  144
VECM                   60 mse_r  1.2021 mse_p    0.04 wr  0.6528 wrp    0.00 len  144
VECM                   84 mse_r  1.2454 mse_p    0.03 wr  0.6528 wrp    0.00 len  144
VECM                  108 mse_r  1.3001 mse_p    0.02 wr  0.6944 wrp    0.00 len  144
VECM                  132 mse_r  1.6191 mse_p    0.00 wr  0.5208 wrp    0.31 len  144
VECM                  156 mse_r  1.1719 mse_p    0.01 wr  0.4583 wrp    0.16 len  144
VECM                  180 mse_r  1.1977 mse_p    0.00 wr  0.4653 wrp    0.20 len  144
VECM                  204 mse_r  1.1929 mse_p    0.01 wr  0.5139 wrp    0.37 len  144
VECM                  228 mse_r  1.1638 mse_p    0.00 wr  0.5208 wrp    0.31 len  144
VECM                  252 mse_r  1.0586 mse_p    0.04 wr  0.5417 wrp    0.16 len  144
VECM                  276 mse_r  1.0255 mse_p    0.11 

In [21]:
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)        
nn=len(tsd.loc[pstart:pend])
interval=1
model='VECM'
oosp=12
for isp in range(oosp*2,oosp*200,oosp*2):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
        performance_report(real0,real9,pred)

VECM                   24 mse_r  1.7938 mse_p    0.11 wr  0.5538 wrp    0.11 len  130
VECM                   48 mse_r  1.1300 mse_p    0.01 wr  0.5154 wrp    0.36 len  130
VECM                   72 mse_r  1.0845 mse_p    0.00 wr  0.4692 wrp    0.24 len  130
VECM                   96 mse_r  1.0882 mse_p    0.00 wr  0.4462 wrp    0.11 len  130
VECM                  120 mse_r  1.0974 mse_p    0.00 wr  0.3462 wrp    0.00 len  130
VECM                  144 mse_r  1.1602 mse_p    0.00 wr  0.3615 wrp    0.00 len  130
VECM                  168 mse_r  0.9904 mse_p    0.40 wr  0.4231 wrp    0.04 len  130
VECM                  192 mse_r  0.9860 mse_p    0.38 wr  0.5154 wrp    0.36 len  130
VECM                  216 mse_r  0.9937 mse_p    0.44 wr  0.5077 wrp    0.43 len  130
VECM                  240 mse_r  0.9750 mse_p    0.23 wr  0.6154 wrp    0.00 len  130
VECM                  264 mse_r  0.9568 mse_p    0.02 wr  0.6077 wrp    0.01 len  130
VECM                  288 mse_r  0.9863 mse_p    0.09 

In [22]:
pstart=datetime.date(1983,1,1)
pend=datetime.date(1994,12,31)        
nn=len(tsd.loc[pstart:pend])
interval=1
model='VECM'
oosp=60
for isp in range(oosp*1,oosp*60,6):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'li')
        performance_report(real0,real9,pred)

VECM                   60 mse_r  1.1527 mse_p    0.04 wr  0.4286 wrp    0.10 len   84
VECM                   66 mse_r  1.1705 mse_p    0.02 wr  0.4286 wrp    0.10 len   84
VECM                   72 mse_r  1.1186 mse_p    0.02 wr  0.4405 wrp    0.14 len   84
VECM                   78 mse_r  1.1308 mse_p    0.03 wr  0.5476 wrp    0.19 len   84
VECM                   84 mse_r  1.1178 mse_p    0.03 wr  0.5476 wrp    0.19 len   84
VECM                   90 mse_r  1.0389 mse_p    0.01 wr  0.6071 wrp    0.02 len   84
VECM                   96 mse_r  1.0200 mse_p    0.04 wr  0.6667 wrp    0.00 len   84
VECM                  102 mse_r  1.0024 mse_p    0.30 wr  0.6429 wrp    0.00 len   84
VECM                  108 mse_r  0.9808 mse_p    0.01 wr  0.7262 wrp    0.00 len   84
VECM                  114 mse_r  0.9595 mse_p    0.00 wr  0.7381 wrp    0.00 len   84
VECM                  120 mse_r  0.9253 mse_p    0.00 wr  0.7262 wrp    0.00 len   84
VECM                  126 mse_r  0.9278 mse_p    0.00 

In [23]:
pstart=datetime.date(1995,1,1)
pend=datetime.date(2007,12,31)        
nn=len(tsd.loc[pstart:pend])
interval=1
model='VECM'
oosp=60
for isp in range(oosp*2,oosp*60,12):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
        performance_report(real0,real9,pred)

VECM                  120 mse_r  7.8277 mse_p    0.00 wr  0.5833 wrp    0.05 len   96
VECM                  132 mse_r 11.4680 mse_p    0.00 wr  0.5000 wrp    0.50 len   96
VECM                  144 mse_r  9.6368 mse_p    0.01 wr  0.4167 wrp    0.05 len   96
VECM                  156 mse_r  7.4618 mse_p    0.01 wr  0.3646 wrp    0.00 len   96
VECM                  168 mse_r  7.7220 mse_p    0.00 wr  0.3333 wrp    0.00 len   96
VECM                  180 mse_r  7.9971 mse_p    0.00 wr  0.4167 wrp    0.05 len   96
VECM                  192 mse_r  3.5851 mse_p    0.00 wr  0.4583 wrp    0.21 len   96
VECM                  204 mse_r  5.6390 mse_p    0.05 wr  0.4271 wrp    0.08 len   96
VECM                  216 mse_r  5.6170 mse_p    0.01 wr  0.4896 wrp    0.42 len   96
VECM                  228 mse_r  3.1456 mse_p    0.03 wr  0.4688 wrp    0.27 len   96
VECM                  240 mse_r  2.1996 mse_p    0.01 wr  0.4792 wrp    0.34 len   96
VECM                  252 mse_r  1.6120 mse_p    0.04 

In [24]:
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)        
nn=len(tsd.loc[pstart:pend])
interval=1
model='VECM'
oosp=60
for isp in range(oosp*1,oosp*60,18):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
        performance_report(real0,real9,pred)

VECM                   60 mse_r  0.9894 mse_p    0.14 wr  0.5366 wrp    0.25 len   82
VECM                   78 mse_r  0.9085 mse_p    0.00 wr  0.7073 wrp    0.00 len   82
VECM                   96 mse_r  0.9680 mse_p    0.02 wr  0.6341 wrp    0.01 len   82
VECM                  114 mse_r  0.9435 mse_p    0.01 wr  0.6220 wrp    0.01 len   82
VECM                  132 mse_r  0.8957 mse_p    0.00 wr  0.6585 wrp    0.00 len   82
VECM                  150 mse_r  0.7698 mse_p    0.00 wr  0.6707 wrp    0.00 len   82
VECM                  168 mse_r  0.7079 mse_p    0.00 wr  0.6951 wrp    0.00 len   82
VECM                  186 mse_r  0.8290 mse_p    0.00 wr  0.7439 wrp    0.00 len   82
VECM                  204 mse_r  0.8080 mse_p    0.00 wr  0.6463 wrp    0.00 len   82
VECM                  222 mse_r  0.8691 mse_p    0.00 wr  0.3902 wrp    0.02 len   82
VECM                  240 mse_r  1.0892 mse_p    0.00 wr  0.2317 wrp    0.00 len   82
VECM                  258 mse_r  1.1048 mse_p    0.00 